**Секція 1. Логістична регресія з нуля.**

Будемо крок за кроком будувати модель лог регресії з нуля для передбачення, чи буде врожай більше за 80 яблук (задача подібна до лекційної, але на класифікацію).

Давайте нагадаємо основні формули для логістичної регресії.

### Функція гіпотези - обчислення передбачення у логістичній регресії:

$$
\hat{y} = \sigma(x W^T + b) = \frac{1}{1 + e^{-(x W^T + b)}}
$$

Де:
- $ \hat{y} $ — це ймовірність "позитивного" класу.
- $ x $ — це вектор (або матриця для набору прикладів) вхідних даних.
- $ W $ — це вектор (або матриця) вагових коефіцієнтів моделі.
- $ b $ — це зміщення (bias).
- $ \sigma(z) $ — це сигмоїдна функція активації.

### Як обчислюється сигмоїдна функція:

Сигмоїдна функція $ \sigma(z) $ має вигляд:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

Ця функція перетворює будь-яке дійсне значення $ z $ в інтервал від 0 до 1, що дозволяє інтерпретувати вихід як ймовірність для логістичної регресії.

### Формула функції втрат для логістичної регресії (бінарна крос-ентропія):

Функція втрат крос-ентропії оцінює, наскільки добре модель передбачає класи, порівнюючи передбачені ймовірності $ \hat{y} $ із справжніми мітками $ y $. Формула наступна:

$$
L(y, \hat{y}) = - \left[ y \cdot \log(\hat{y}) + (1 - y) \cdot \log(1 - \hat{y}) \right]
$$

Де:
- $ y $ — це справжнє значення (мітка класу, 0 або 1).
- $ \hat{y} $ — це передбачене значення (ймовірність).



1.
Тут вже наведений код для ініціювання набору даних в форматі numpy. Перетворіть `inputs`, `targets` на `torch` тензори. Виведіть результат на екран.

In [286]:
# !pip install torch

In [288]:
import torch
import numpy as np

In [290]:
# Вхідні дані (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

# Таргети (apples > 80)
targets = np.array([[0],
                    [1],
                    [1],
                    [0],
                    [1]], dtype='float32')

In [292]:
inputs = torch.tensor(inputs)
targets = torch.tensor(targets)

inputs, targets

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]),
 tensor([[0.],
         [1.],
         [1.],
         [0.],
         [1.]]))

2. Ініціюйте ваги `w`, `b` для моделі логістичної регресії потрібної форми зважаючи на розмірності даних випадковими значеннями з нормального розподілу. Лишаю тут код для фіксації `random_seed`.

In [295]:
torch.random.manual_seed(1)

In [297]:
w = torch.randn(1, 3, dtype=torch.float32, requires_grad=True)  # 1 вихід -  прогноз одного числа для яблук, 3 ознаки - temp, rainfall, humidity
b = torch.randn(1, dtype=torch.float32, requires_grad=True)

w, b

(tensor([[0.6614, 0.2669, 0.0617]], requires_grad=True),
 tensor([0.6213], requires_grad=True))

3. Напишіть функцію `model`, яка буде обчислювати функцію гіпотези в логістичній регресії і дозволяти робити передбачення на основі введеного рядка даних і коефіцієнтів в змінних `w`, `b`.

  **Важливий момент**, що функція `model` робить обчислення на `torch.tensors`, тож для математичних обчислень використовуємо фукнціонал `torch`, наприклад:
  - обчсилення $e^x$: `torch.exp(x)`
  - обчсилення $log(x)$: `torch.log(x)`
  - обчислення середнього значення вектору `x`: `torch.mean(x)`

  Використайте функцію `model` для обчислення передбачень з поточними значеннями `w`, `b`.Виведіть результат обчислень на екран.

  Проаналізуйте передбачення. Чи не викликають вони у вас підозр? І якщо викликають, то чим це може бути зумовлено?

In [300]:
# Функція для обчислення гіпотези в логістичній регресії
def model(x, w, b):
    linear_output = x @ w.t() + b
    predictions = 1 / (1 + torch.exp(-linear_output))
    return predictions

In [302]:
prediction = model(inputs, w, b)
prediction

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]], grad_fn=<MulBackward0>)

У нас у всіх випадках передбачає більше за 80 яблук врожайності, що, звісно, виглядає упереджено.

4. Напишіть функцію `binary_cross_entropy`, яка приймає на вхід передбачення моделі `predicted_probs` та справжні мітки в даних `true_labels` і обчислює значення втрат (loss)  за формулою бінарної крос-ентропії для кожного екземпляра та вертає середні втрати по всьому набору даних.
  Використайте функцію `binary_cross_entropy` для обчислення втрат для поточних передбачень моделі.

In [306]:
# def binary_cross_entropy(predicted_probs, true_labels):
#     epsilon = 1e-7     # Запобігаємо логарифмуванню дуже малих значень (додаємо epsilon)
#     predicted_probs = torch.clamp(predicted_probs, min=epsilon, max=1-epsilon)
#     bce_loss = - (true_labels * torch.log(predicted_probs) + (1 - true_labels) * torch.log(1 - predicted_probs))
#     return torch.mean(bce_loss)

In [308]:
def binary_cross_entropy(predicted_probs, true_labels):
    bce_loss = - (true_labels * torch.log(predicted_probs) + (1 - true_labels) * torch.log(1 - predicted_probs))
    return torch.mean(bce_loss)

In [310]:
loss = binary_cross_entropy(prediction, targets)
loss

tensor(nan, grad_fn=<MeanBackward0>)

5. Зробіть зворотнє поширення помилки і виведіть градієнти за параметрами `w`, `b`. Проаналізуйте їх значення. Як гадаєте, чому вони саме такі?

In [313]:
loss.backward()

In [315]:
w, w.grad

(tensor([[0.6614, 0.2669, 0.0617]], requires_grad=True),
 tensor([[nan, nan, nan]]))

In [317]:
b, b.grad

(tensor([0.6213], requires_grad=True), tensor([nan]))

отримали nan. Значення передбачень були занадто близькими до 1, що спричинило проблему під час обчислення логарифмів

**Що сталось?**

В цій задачі, коли ми ініціювали значення випадковими значеннями з нормального розподілу, насправді ці значення не були дуже гарними стартовими значеннями і привели до того, що градієнти стали дуже малими або навіть рівними нулю (це призводить до того, що градієнти "зникають"), і відповідно при оновленні ваг у нас не буде нічого змінюватись. Це називається `gradient vanishing`. Це відбувається через **насичення сигмоїдної функції активації.**

У нашій задачі ми використовуємо сигмоїдну функцію активації, яка має такий вигляд:

   $$
   \sigma(z) = \frac{1}{1 + e^{-z}}
   $$


Коли значення $z$ дуже велике або дуже мале, сигмоїдна функція починає "насичуватись". Це означає, що для великих позитивних $z$ сигмоїда наближається до 1, а для великих негативних — до 0. В цих діапазонах градієнти починають стрімко зменшуватись і наближаються до нуля (бо градієнт - це похідна, похідна на проміжку функції, де вона паралельна осі ОХ, дорівнює 0), що робить оновлення ваг неможливим.

![](https://editor.analyticsvidhya.com/uploads/27889vaegp.png)

У логістичній регресії $ z = x \cdot w + b $. Якщо ваги $w, b$ - великі, значення $z$ також буде великим, і сигмоїда перейде в насичену область, де градієнти дуже малі.

Саме це сталося в нашій задачі, де великі випадкові значення ваг викликали насичення сигмоїдної функції. Це в свою чергу призводить до того, що під час зворотного поширення помилки (backpropagation) модель оновлює ваги дуже повільно або зовсім не оновлює. Це називається проблемою **зникнення градієнтів** (gradient vanishing problem).

**Що ж робити?**
Ініціювати ваги маленькими значеннями навколо нуля. Наприклад ми можемо просто в існуючій ініціалізації ваги розділити на 1000. Можна також використати інший спосіб ініціалізації вагів - інформація про це [тут](https://www.geeksforgeeks.org/initialize-weights-in-pytorch/).

Як це робити - показую нижче. **Виконайте код та знову обчисліть передбачення, лосс і виведіть градієнти.**

А я пишу пояснення, чому просто не зробити

```
w = torch.randn(1, 3, requires_grad=True)/1000
b = torch.randn(1, requires_grad=True)/1000
```

Нам потрібно, аби тензори вагів були листовими (leaf tensors).

1. **Що таке листовий тензор**
Листовий тензор — це тензор, який був створений користувачем безпосередньо і з якого починається обчислювальний граф. Якщо такий тензор має `requires_grad=True`, PyTorch буде відслідковувати всі операції, виконані над ним, щоб правильно обчислювати градієнти під час навчання.

2. **Чому ми використовуємо `w.data` замість звичайних операцій**
Якщо ми просто виконали б операції, такі як `(w - 0.5) / 100`, ми б отримали **новий тензор**, який вже не був би листовим тензором, оскільки ці операції створюють **новий** тензор, а не модифікують існуючий.

  Проте, щоб залишити наші тензори ваги `w` та зміщення `b` листовими і продовжити можливість відстеження градієнтів під час тренування, ми використовуємо атрибут `.data`. Цей атрибут дозволяє **виконувати операції in-place (прямо на існуючому тензорі)** без зміни самого об'єкта тензора. Отже, тензор залишається листовим, і PyTorch може коректно обчислювати його градієнти.

3. **Чому важливо залишити тензор листовим**
Якщо тензор більше не є листовим (наприклад, через проведення операцій, що створюють нові тензори), ви не зможете отримати градієнти за допомогою `w.grad` чи `b.grad` після виклику `loss.backward()`. Це може призвести до втрати можливості оновлення параметрів під час тренування моделі. В нашому випадку ми хочемо, щоб тензори `w` та `b` накопичували градієнти, тому вони повинні залишатись листовими.

**Висновок:**
Ми використовуємо `.data`, щоб виконати операції зміни значень на ваги і зміщення **in-place**, залишаючи їх листовими тензорами, які можуть накопичувати градієнти під час навчання. Це дозволяє коректно працювати механізму зворотного поширення помилки (backpropagation) і оновлювати ваги моделі.

5. Виконайте код та знову обчисліть передбачення, лосс і знайдіть градієнти та виведіть всі ці тензори на екран.

In [322]:
torch.random.manual_seed(1)
w = torch.randn(1, 3, requires_grad=True)  # Листовий тензор
b = torch.randn(1, requires_grad=True)     # Листовий тензор

# in-place операції
w.data = w.data / 1000
b.data = b.data / 1000

In [324]:
prediction = model(inputs, w, b)
prediction

tensor([[0.5174],
        [0.5220],
        [0.5244],
        [0.5204],
        [0.5190]], grad_fn=<MulBackward0>)

In [326]:
loss = binary_cross_entropy(prediction, targets)
loss

tensor(0.6829, grad_fn=<MeanBackward0>)

In [328]:
loss.backward()

In [330]:
w, w.grad

(tensor([[6.6135e-04, 2.6692e-04, 6.1677e-05]], requires_grad=True),
 tensor([[ -5.4417, -18.9853, -10.0682]]))

In [332]:
b, b.grad

(tensor([0.0006], requires_grad=True), tensor([-0.0794]))

6. Напишіть алгоритм градієнтного спуску, який буде навчати модель з використанням написаних раніше функцій і виконуючи оновлення ваг. Алгоритм має включати наступні кроки:

  1. Генерація прогнозів
  2. Обчислення втрат
  3. Обчислення градієнтів (gradients) loss-фукнції відносно ваг і зсувів
  4. Налаштування ваг шляхом віднімання невеликої величини, пропорційної градієнту (`learning_rate` домножений на градієнт)
  5. Скидання градієнтів на нуль

Виконайте градієнтний спуск протягом 1000 епох, обчисліть фінальні передбачення і проаналізуйте, чи вони точні?

In [335]:
learning_rate = 1e-5
epochs = 1000

for epoch in range(epochs):
    predictions = model(inputs, w, b)

    loss = binary_cross_entropy(predictions, targets)

    loss.backward()

    with torch.no_grad():
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad

    w.grad.zero_()
    b.grad.zero_()

    if (epoch+1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

final_predictions = model(inputs, w, b)
print("Фінальні передбачення:\n", final_predictions)

final_binary_predictions = (final_predictions >= 0.5).float()
print("Фінальні бінарні передбачення:\n", final_binary_predictions)

accuracy = (final_binary_predictions == targets).float().mean()
print(f"Точність моделі: {accuracy.item() * 100:.2f}%")

Epoch 100/1000, Loss: 0.5625792145729065
Epoch 200/1000, Loss: 0.5070590972900391
Epoch 300/1000, Loss: 0.4650115370750427
Epoch 400/1000, Loss: 0.4326744079589844
Epoch 500/1000, Loss: 0.40733838081359863
Epoch 600/1000, Loss: 0.3871012330055237
Epoch 700/1000, Loss: 0.37063437700271606
Epoch 800/1000, Loss: 0.3570035398006439
Epoch 900/1000, Loss: 0.3455430269241333
Epoch 1000/1000, Loss: 0.3357715606689453
Фінальні передбачення:
 tensor([[0.5777],
        [0.6685],
        [0.9113],
        [0.1616],
        [0.8653]], grad_fn=<MulBackward0>)
Фінальні бінарні передбачення:
 tensor([[1.],
        [1.],
        [1.],
        [0.],
        [1.]])
Точність моделі: 80.00%


Вийшла доволі непогана модель з точністю 80%.

**Секція 2. Створення лог регресії з використанням функціоналу `torch.nn`.**

Давайте повторно реалізуємо ту ж модель, використовуючи деякі вбудовані функції та класи з PyTorch.

Даних у нас буде побільше - тож, визначаємо нові масиви.

In [339]:
# Вхідні дані (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

# Таргети (apples > 80)
targets = np.array([[0],
                    [1],
                    [1],
                    [0],
                    [1],
                    [0],
                    [1],
                    [1],
                    [0],
                    [1],
                    [0],
                    [1],
                    [1],
                    [0],
                    [1]], dtype='float32')

7. Завантажте вхідні дані та мітки в PyTorch тензори та з них створіть датасет, який поєднує вхідні дані з мітками, використовуючи клас `TensorDataset`. Виведіть перші 3 елементи в датасеті.



In [342]:
from torch.utils.data import TensorDataset

inputs = torch.tensor(inputs, dtype=torch.float32)
targets = torch.tensor(targets, dtype=torch.float32)

dataset = TensorDataset(inputs_tensor, targets_tensor)

In [344]:
dataset[:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[0.],
         [1.],
         [1.]]))

8. Визначте data loader з класом **DataLoader** для підготовленого датасету `train_ds`, встановіть розмір батчу на 5 та увімкніть перемішування даних для ефективного навчання моделі. Виведіть перший елемент в дата лоадері.

In [347]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=dataset, batch_size=5, shuffle=True)

first_batch = next(iter(train_loader))

first_batch[0][0], first_batch[1][0]

(tensor([73., 67., 43.]), tensor([0.]))

9. Створіть клас `LogReg` для логістичної регресії, наслідуючи модуль `torch.nn.Module` за прикладом в лекції (в частині про FeedForward мережі).

  У нас модель складається з лінійної комбінації вхідних значень і застосування фукнції сигмоїда. Тож, нейромережа буде складатись з лінійного шару `nn.Linear` і використання активації `nn.Sigmid`. У створеному класі мають бути реалізовані методи `__init__` з ініціалізацією шарів і метод `forward` для виконання прямого проходу моделі через лінійний шар і функцію активації.

  Створіть екземпляр класу `LogReg` в змінній `model`.

In [350]:
import torch.nn as nn

class LogReg(nn.Module):
    def __init__(self, input_size, output_size):
        """
        Ініціалізація класу логістичної регресії.
        
        :param input_size: Кількість ознак (features) на вхід
        :param output_size: Кількість виходів (output), зазвичай 1 для бінарної класифікації
        """
        super(LogReg, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        """
        Прямий прохід через модель.
        
        :param x: Вхідні дані (тензор)
        :return: Результати моделі (передбачення)
        """
        linear_output = self.linear(x)
        return self.sigmoid(linear_output)

input_size = 3  
output_size = 1

model = LogReg(input_size, output_size)

model

LogReg(
  (linear): Linear(in_features=3, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

10. Задайте оптимізатор `Stockastic Gradient Descent` в змінній `opt` для навчання моделі логістичної регресії. А також визначіть в змінній `loss` функцію втрат `binary_cross_entropy` з модуля `torch.nn.functional` для обчислення втрат моделі. Обчисліть втрати для поточних передбачень і міток, а потім виведіть їх. Зробіть висновок, чи моделі вдалось навчитись?

In [353]:
import torch.nn.functional as F

# Параметри
learning_rate = 1e-5
epochs = 1000
batch_size = 5

# Створення оптимізатора Stockastic Gradient Descent (SGD)
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Функція втрат
loss_fn = F.binary_cross_entropy

loss = loss_fn(model(inputs), targets)
loss

tensor(7.6312, grad_fn=<BinaryCrossEntropyBackward0>)

11. Візьміть з лекції функцію для тренування моделі з відстеженням значень втрат і навчіть щойно визначену модель на 1000 епохах. Виведіть після цього графік зміни loss, фінальні передбачення і значення таргетів.

In [355]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    model.train() 
    running_loss = 0.0

    for inputs_batch, targets_batch in train_loader:
        opt.zero_grad()  
        
        predictions = model(inputs_batch)
        
        # Обчислюємо втрати
        loss = compute_loss(predictions, targets_batch)
        running_loss += loss.item()
        
        loss.backward()
        opt.step()
    

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader)}")

model.eval() 
with torch.no_grad():
    final_predictions = model(torch.tensor(inputs, dtype=torch.float32))
    final_loss = compute_loss(final_predictions, torch.tensor(targets, dtype=torch.float32))

print(f"Фінальна втрата: {final_loss.item()}")

Epoch [1/1000], Loss: 7.590260028839111
Epoch [2/1000], Loss: 7.466097990671794
Epoch [3/1000], Loss: 7.347989400227864
Epoch [4/1000], Loss: 7.238023122151692
Epoch [5/1000], Loss: 7.1360955238342285
Epoch [6/1000], Loss: 7.012691020965576
Epoch [7/1000], Loss: 6.917130470275879
Epoch [8/1000], Loss: 6.817821979522705
Epoch [9/1000], Loss: 6.729780991872151
Epoch [10/1000], Loss: 6.640044689178467
Epoch [11/1000], Loss: 6.563978354136149
Epoch [12/1000], Loss: 6.491995493570964
Epoch [13/1000], Loss: 6.443535486857097
Epoch [14/1000], Loss: 6.355709075927734
Epoch [15/1000], Loss: 6.294747670491536
Epoch [16/1000], Loss: 6.2434327602386475
Epoch [17/1000], Loss: 6.195674498875936
Epoch [18/1000], Loss: 6.143290996551514
Epoch [19/1000], Loss: 6.097677071889241
Epoch [20/1000], Loss: 6.048301617304484
Epoch [21/1000], Loss: 6.012735366821289
Epoch [22/1000], Loss: 5.973509788513184
Epoch [23/1000], Loss: 5.935581207275391
Epoch [24/1000], Loss: 5.905618826548259
Epoch [25/1000], Loss: 

/var/folders/4x/c_v1qrzn0_z359h7ycv_jvsc0000gn/T/ipykernel_88191/3628309636.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  final_predictions = model(torch.tensor(inputs, dtype=torch.float32))
/var/folders/4x/c_v1qrzn0_z359h7ycv_jvsc0000gn/T/ipykernel_88191/3628309636.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  final_loss = compute_loss(final_predictions, torch.tensor(targets, dtype=torch.float32))
